<a href="https://colab.research.google.com/github/wildlifeai/pepeketua_zooniverse/blob/main/check_pepeketua_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the scripts to map the identification data of Archey's frogs (excel spreadsheet) with the actual photos of the frogs.

#Requirements

### Load required libraries

Load generic libraries

In [ ]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 967 kB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import io, os
import zipfile
import json
import pandas as pd
import numpy as np

from google.colab import drive
from datetime import date

#Load the path of the photos

###Add shortcuts to the compressed photos

To download the photos of the frogs into this Google Colab you first need to add shortcuts in your Google drive to the [five zipped folders](https://drive.google.com/file/d/1XXSrATFX1l-J0CUE4m6UfoOBp9zv3XOr/view?usp=sharing) with the photos. 

To add the shortcuts:
* go to the "Shared with me" section in your Google drive,
* find the five zipped folders,
* click on "Add shorcut to Drive" and
* save the shortcuts (we created a folder called "frog_photos" and saved them there).

*Specify* the folder in your Google drive where you saved the shortcuts to the photos (in our case "frog_photos").

In [ ]:
dir_shortcuts = "/content/drive/MyDrive/Projects/pepeketua_id/frog_photos/"

*If you can't access the five zipped folders please [email Victor](victor@wildlife.ai). 

###Load the zipped files

To download the five zip folders with the photos you will need to grant access to the Google file stream. 



In [ ]:
# Mount the drive in colab
drive.mount('/content/drive/')

# Load the five zipped files
whareorino_a = zipfile.ZipFile(dir_shortcuts + "whareorino_a.zip", 'r')
whareorino_b = zipfile.ZipFile(dir_shortcuts + "whareorino_b.zip", 'r')
whareorino_c = zipfile.ZipFile(dir_shortcuts + "whareorino_c.zip", 'r')
whareorino_d = zipfile.ZipFile(dir_shortcuts + "whareorino_d.zip", 'r')
pukeokahu = zipfile.ZipFile(dir_shortcuts + "pukeokahu.zip", 'r')

# Extract the filepath of the photos of individual frogs
zips = [whareorino_a, whareorino_b, whareorino_c, whareorino_d, pukeokahu]
pdList = []

for zip_file in zips:
  zip_pd = pd.DataFrame(
      [x for x in zip_file.namelist() if 'Individual Frogs' in x and not x.endswith(('.db','/','Store'))]
      )
  pdList.append(zip_pd)

# Combine the file paths of the five grids into a single data frame
frog_df = pd.concat(pdList)


Mounted at /content/drive/


## Prepare information related to the photos

In [ ]:
# Rename the column of df
frog_df = frog_df.rename(columns={0: "filepath"})

# Add new columns using directory and filename information
directories = frog_df['filepath'].str.split("/", n = 4, expand = True)

# Add the grid, frog_id, filename, and capture cols 
frog_df["grid"] = directories[0]
frog_df["frog_id"] = directories[2] 
frog_df["filename"] = directories[3]  
frog_df["Capture photo code"] = frog_df["filename"].str.split(".",1, expand = True)[0] 
frog_df["capture"] = frog_df["filename"].str.split(".",1, expand = True)[0].str.replace('_', '-').str.rsplit("-",1, expand = True)[1]

# Manually filter out non-standard photos
# frog_df = frog_df[~frog_df['filename'].str.contains(("Picture|IMG|#"))]


# Load the excel spreadsheets

###Read the spreadsheets with frog capture information

In [ ]:
# whareorino_df = pd.read_excel("/content/drive/MyDrive/Projects/pepeketua_id/Whareorino update.xls",
#               sheet_name=['Grid A','Grid B','Grid C','Grid D']) 
# pukeokahu_df = pd.read_excel("/content/drive/MyDrive/Projects/pepeketua_id/Pukeokahu update.xls",
#               sheet_name=['MR Data']) 

whareorino_df = pd.read_excel("/content/drive/MyDrive/Projects/pepeketua_id/Whareorino frog monitoring data 2005 onwards CURRENT FILE - DOCDM-106978.xls",
              sheet_name=['Grid A','Grid B','Grid C','Grid D']) 
pukeokahu_df = pd.read_excel("/content/drive/MyDrive/Projects/pepeketua_id/Pukeokahu Monitoring Data 2006 onwards - DOCDM-95563.xls",
              sheet_name=['MR Data'])


### Add grid column to the frog capture info

In [ ]:
whareorino_df['Grid A']["grid"]="Grid A"
whareorino_df['Grid B']["grid"]="Grid B"
whareorino_df['Grid C']["grid"]="Grid C"
whareorino_df['Grid D']["grid"]="Grid D"
pukeokahu_df['MR Data']["grid"]="Pukeokahu Frog Monitoring"

In [ ]:
# Combine datasets
frog_id_df = pd.concat([whareorino_df['Grid A'],
                   whareorino_df['Grid B'],
                   whareorino_df['Grid C'],
                   whareorino_df['Grid D'],
                   pukeokahu_df['MR Data'],]
                   )

### Limit the df to frog identifications older than 2020

In [ ]:
import datetime
# Select rows with valid dates
valid_frog_id_df = frog_id_df[(frog_id_df['Date'].notnull())&(frog_id_df['Date'] != "Date")]

# Filter observations older than 2020
valid_frog_id_df = valid_frog_id_df[valid_frog_id_df['Date'].astype('datetime64[ns]')<datetime.datetime(year=2020,month=1,day=1)]

### Remove manual typos and faulty entries

In [ ]:
wrong_capture_id = ['GRID SEARCHED BUT ZERO FROGS FOUND =(', 'hochstetter']
valid_frog_id_df = valid_frog_id_df[~valid_frog_id_df['Capture #'].isin(wrong_capture_id)]

# Remove empty capture 
valid_frog_id_df = valid_frog_id_df.dropna(subset=['Capture #'])

# Remove empty capture 
valid_frog_id_df = valid_frog_id_df.dropna(subset=['Capture photo code'])

In [ ]:
# Number of photos identified per grid
valid_frog_id_df.groupby(['grid'])['grid'].count()

grid
Grid A                        806
Grid B                       1446
Grid C                       2591
Grid D                       2788
Pukeokahu Frog Monitoring     851
Name: grid, dtype: int64

# Map the photos with the frog identification data

In [ ]:
df = valid_frog_id_df.merge(frog_df, on=['Capture photo code','grid'], how='left')

In [ ]:
df.groupby(['grid'])['grid'].count()

grid
Grid A                        806
Grid B                       1448
Grid C                       2598
Grid D                       2796
Pukeokahu Frog Monitoring     852
Name: grid, dtype: int64

# Work in progress to clean and tidy out the data

## Find out duplicated photos

In [ ]:
if df[df.duplicated(['Capture photo code','grid'],keep=False)][['Capture #','grid','Capture photo code',"filepath"]].shape[0]>0:
  print("There are", df[df.duplicated(['Capture photo code','grid'],keep=False)][['Capture #','grid','Capture photo code',"filepath"]].shape[0], "duplicates")
  print(df[df.duplicated(['Capture photo code','grid'],keep=False)][['Capture #','grid','Capture photo code',"filepath"]])
  df[df.duplicated(['Capture photo code','grid'],keep=False)][['Capture #','grid','Capture photo code',"filepath"]].to_csv("duplicated_frog_photos.csv")


## Find out identifications that can't be mapped to a photo (missing filepaths)

In [ ]:
# Missing filepaths per grid
df[df.columns.difference(['grid'])].isnull().groupby(df.grid).sum().astype(int)["filepath"]

grid
Grid A                        94
Grid B                       147
Grid C                       357
Grid D                       235
Pukeokahu Frog Monitoring    114
Name: filepath, dtype: int64

In [ ]:
import numpy as np

# Rename original photo code
df = df.rename(columns={"Capture photo code":"Original Capture photo code"})

# Modify 'Capture photo code' using the marks and Capture # of those photos unable to be located
df['Capture photo code'] = np.where(df["filepath"].isna(), 
                                    df['Back left mark'].astype(str).apply(lambda x: '_' if '?' in x else x) + 
                                    df['Back right mark'].astype(str).apply(lambda x: '_' if '?' in x else x) +
                                    df['Face left mark'].astype(str).apply(lambda x: '_' if '?' in x else x) +
                                    df['Face right mark'].astype(str).apply(lambda x: '_' if '?' in x else x) +
                                    '-' + 
                                    df['Capture #'].astype(int).astype(str), 
                                    df['Original Capture photo code'])


In [ ]:
# Add filepath of the photos to each frog identification again with the updated 'Capture photo code'
df = df.drop(columns=list(set(list(frog_df.columns)) - set(['Capture photo code','grid']))).merge(frog_df, on=['Capture photo code','grid'], how='left')
df[df.columns.difference(['grid'])].isnull().groupby(df.grid).sum().astype(int)["filepath"]

grid
Grid A                        32
Grid B                        62
Grid C                       245
Grid D                       153
Pukeokahu Frog Monitoring    110
Name: filepath, dtype: int64

In [ ]:
# Modify 'Capture photo code' using the marks and Capture # of those photos unable to be located
df['Capture photo code'] = np.where(df["filepath"].isna(), 
                                    df['Back left mark'].astype(str).apply(lambda x: '0' if '?' in x else x) + 
                                    df['Back right mark'].astype(str).apply(lambda x: '0' if '?' in x else x) +
                                    df['Face left mark'].astype(str).apply(lambda x: '0' if '?' in x else x) +
                                    df['Face right mark'].astype(str).apply(lambda x: '0' if '?' in x else x) +
                                    '-' + 
                                    df['Capture #'].astype(int).astype(str), 
                                    df['Capture photo code'])


In [ ]:
# Add filepath of the photos to each frog identification again with the updated 'Capture photo code'
df = df.drop(columns=list(set(list(frog_df.columns)) - set(['Capture photo code','grid']))).merge(frog_df, on=['Capture photo code','grid'], how='left')
df[df.columns.difference(['grid'])].isnull().groupby(df.grid).sum().astype(int)["filepath"]

grid
Grid A                        32
Grid B                        62
Grid C                       241
Grid D                       151
Pukeokahu Frog Monitoring    109
Name: filepath, dtype: int64

In [ ]:
# Modify 'Capture photo code' using the marks and Capture # of those photos unable to be located
df['Capture photo code'] = np.where(df["filepath"].isna(), 
                                    df['Back left mark'].astype(str).apply(lambda x: '1' if '?' in x else x) + 
                                    df['Back right mark'].astype(str).apply(lambda x: '1' if '?' in x else x) +
                                    df['Face left mark'].astype(str).apply(lambda x: '1' if '?' in x else x) +
                                    df['Face right mark'].astype(str).apply(lambda x: '1' if '?' in x else x) +
                                    '-' + 
                                    df['Capture #'].astype(int).astype(str), 
                                    df['Capture photo code'])


In [ ]:
# Add filepath of the photos to each frog identification again with the updated 'Capture photo code'
df = df.drop(columns=list(set(list(frog_df.columns)) - set(['Capture photo code','grid']))).merge(frog_df, on=['Capture photo code','grid'], how='left')
df[df.columns.difference(['grid'])].isnull().groupby(df.grid).sum().astype(int)["filepath"]

grid
Grid A                        32
Grid B                        62
Grid C                       240
Grid D                       149
Pukeokahu Frog Monitoring    107
Name: filepath, dtype: int64

In [ ]:
df = df.rename(columns={"Capture photo code":"updated Capture photo code"})

df = df.drop_duplicates(['Capture #', 'grid'])


In [ ]:
new_df = frog_id_df.merge(df[['Capture #', 'grid', 'updated Capture photo code']], 
                          on=['Capture #', 'grid'], 
                          how='left')

# make sure if empty original values are used
new_df["updated Capture photo code"] = np.where(new_df["updated Capture photo code"].isna(), 
                                    new_df['Capture photo code'], 
                                    new_df["updated Capture photo code"])

new_df["updated Capture photo code"] = np.where(new_df["updated Capture photo code"].isna(), 
                                    new_df['Capture photo code'], 
                                    new_df["updated Capture photo code"])

new_df['different Capture photo Code'] = np.where(new_df["Capture photo code"]==new_df['updated Capture photo code'], 0, 1)



In [ ]:
# Closest match between the Capture photo code and filenames
new_df[new_df['grid']=="Grid A"].drop(columns=['grid']).to_csv("victor_reviewed_grid_a.csv")
new_df[new_df['grid']=="Grid B"].drop(columns=['grid']).to_csv("victor_reviewed_grid_b.csv")
new_df[new_df['grid']=="Grid C"].drop(columns=['grid']).to_csv("victor_reviewed_grid_c.csv")
new_df[new_df['grid']=="Grid D"].drop(columns=['grid']).to_csv("victor_reviewed_grid_d.csv")
new_df[new_df['grid']=="Pukeokahu Frog Monitoring"].drop(columns=['grid']).to_csv("victor_reviewed_pukeokahu.csv")

In [ ]:
# Missing photos
df[(df['grid']=="Grid A")&(df["filepath"].isna())].to_csv("missing_grid_a.csv")
df[(df['grid']=="Grid B")&(df["filepath"].isna())].to_csv("missing_grid_b.csv")
df[(df['grid']=="Grid C")&(df["filepath"].isna())].to_csv("missing_grid_c.csv")
df[(df['grid']=="Grid D")&(df["filepath"].isna())].to_csv("missing_grid_d.csv")
df[(df['grid']=="Pukeokahu Frog Monitoring")&(df["filepath"].isna())].to_csv("missing_pukeokahu.csv")

## Check for consistent column names

In [ ]:
# AB
col_diff = list(set(whareorino_df['Grid A'].columns) - set(whareorino_df['Grid B'].columns))
if col_diff:
  print("Differences between A and B", col_diff)

# BA
col_diff = list(set(whareorino_df['Grid B'].columns) - set(whareorino_df['Grid A'].columns))
if col_diff:
  print("Differences between B and A", col_diff)

# AC
col_diff = list(set(whareorino_df['Grid A'].columns) - set(whareorino_df['Grid C'].columns))
if col_diff:
  print("Differences between A and C", col_diff)

# CA
col_diff = list(set(whareorino_df['Grid C'].columns) - set(whareorino_df['Grid A'].columns))
if col_diff:
  print("Differences between C and A", col_diff)

# AD
col_diff = list(set(whareorino_df['Grid A'].columns) - set(whareorino_df['Grid D'].columns))
if col_diff:
  print("Differences between A and D", col_diff)

# DA
col_diff = list(set(whareorino_df['Grid D'].columns) - set(whareorino_df['Grid A'].columns))
if col_diff:
  print("Differences between D and A", col_diff)

# AP
col_diff = list(set(whareorino_df['Grid A'].columns) - set(pukeokahu_df['MR Data'].columns))
if col_diff:
  print("Differences between A and pukeokahu", col_diff)

# PA
col_diff = list(set(pukeokahu_df['MR Data'].columns) - set(whareorino_df['Grid A'].columns))
if col_diff:
  print("Differences between pukeokahu and A", col_diff)

In [ ]:
# Add filepath info for whareorino_df
whareorino_df_a_complete_df = whareorino_df['Grid A'].merge(frog_df, on=['Capture photo code','grid'], how='left')

whareorino_df_b_complete_df = whareorino_df['Grid B'].merge(frog_df, on=['Capture photo code','grid'], how='left')

whareorino_df_c_complete_df = whareorino_df['Grid C'].merge(frog_df, on=['Capture photo code','grid'], how='left')

whareorino_df_d_complete_df = whareorino_df['Grid D'].merge(frog_df, on=['Capture photo code','grid'], how='left')

# Add filepath info for pukeokahu
pukeokahu_complete_df = pukeokahu_df['MR Data'].merge(frog_df, on=['Capture photo code','grid'], how='left')